<a href="https://colab.research.google.com/github/Airinelizabath/NLP/blob/main/CH6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import python_utils
import re
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [ ]:
import random

2.  Using any of the three classifiers described in this chapter, and any features you
can think of, build the best name gender classifier you can. Begin by splitting the
Names Corpus into three subsets: 500 words for the test set, 500 words for the
dev-test set, and the remaining 6,900 words for the training set. Then, starting with
the example name gender classifier, make incremental improvements. Use the devtest set to check your progress. Once you are satisfied with your classifier, check
its final performance on the test set. How does the performance on the test set
compare to the performance on the dev-test set? Is this what you’d expect?


In [ ]:
from nltk.corpus import names
import random
names = ([(name, 'male') for name in names.words('male.txt')] +
        [(name, 'female') for name in names.words('female.txt')])
random.shuffle(names)

In [ ]:
def gender_features(word):
    return {'last_letter'  : word[-1]}
featuresets = [(gender_features(n), gender) for (n, gender) in names]
train_set, test_set, devtest_set = featuresets[1000:], featuresets[:500], featuresets[500:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.722


In [ ]:
def gender_features(word):
    return {'last_letter'  : word[-1],
            'length'       : len(word),
            'first_letter' : word[0],}
featuresets = [(gender_features(n), gender) for (n, gender) in names]
train_set, test_set, devtest_set = featuresets[1000:], featuresets[:500], featuresets[500:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.748


In [ ]:
def gender_features(word):
    return {'last_letter'  : word[-1],
            'length'       : len(word),
            'first_letter' : word[0],
            'last_2letters': word[-2:],
            'first_2letters': word[:2]}
featuresets = [(gender_features(n), gender) for (n, gender) in names]
train_set, test_set, devtest_set = featuresets[1000:], featuresets[:500], featuresets[500:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.784


In [ ]:
def gender_features(word):
    return {'last_letter'  : word[-1],
            'length'       : len(word),
            'first_letter' : word[0],
            'last_2letters': word[-2:],
            'first_2letters': word[:2],
            'first_vowel'  : [i for i in range(len(word)) 
                              if word[i] in 'AEIOUaeiouy'][0],
            'double_letters': sum([1 for ch in range(len(word) - 1)
                                  if word[ch] == word[ch + 1]])}
            

featuresets = [(gender_features(n), gender) for (n, gender) in names]
train_set, test_set, devtest_set = featuresets[1000:], featuresets[:500], featuresets[500:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.788


In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

0.796


3. The Senseval 2 Corpus contains data intended to train word-sense disambiguation classifiers. It contains data for four words: *hard, interest, line,* and *serve*.
Choose one of these four words, and load the corresponding data:
```
>>> from nltk.corpus import senseval
>>> instances = senseval.instances('hard.pos')
>>> size = int(len(instances) * 0.1)
>>> train_set, test_set = instances[size:], instances[:size]
```
Using this dataset, build a classifier that predicts the correct sense tag for a given
instance. See the corpus HOWTO at http://www.nltk.org/howto for information
on using the instance objects returned by the Senseval 2 Corpus.


In [ ]:
from nltk.corpus import senseval
import random
instances = senseval.instances('hard.pos')

In [ ]:
size = int(len(instances) * 0.1)
#train_set, test_set = instances[size:], instances[:size]
classifier = nltk.NaiveBayesClassifier.train(instances[size:])


In [ ]:
senses = [int(instances[i].senses[0][4]) for i in range(len(instances))]
def hard_features(inst):
    p = inst.position
    return {'left2':  inst.context[p - 2],
            'left1':  inst.context[p - 1],
            'right':  inst.context[p + 1]}


In [ ]:
hard_features(instances[0])

{'left1': ("'s", 'VBZ'), 'left2': ('that', 'DT'), 'right': ('to', 'TO')}

In [ ]:
featuresets = [(hard_features(n), n.senses) for n in instances]
size = int(len(featuresets) * 0.1)
train_set, test_set = instances[size:], instances[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

 4. Using the movie review document classifier discussed in this chapter, generate
a list of the 30 features that the classifier finds to be most informative. Can you
explain why these particular features are informative? Do you find any of them
surprising?

In [ ]:
from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

In [ ]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [ ]:
featuresets = [(document_features(d), c) for (d, c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:

classifier.show_most_informative_features(30)

Most Informative Features
 contains(unimaginative) = True              neg : pos    =      7.7 : 1.0
        contains(turkey) = True              neg : pos    =      7.6 : 1.0
    contains(schumacher) = True              neg : pos    =      7.1 : 1.0
     contains(atrocious) = True              neg : pos    =      7.1 : 1.0
        contains(suvari) = True              neg : pos    =      7.1 : 1.0
          contains(mena) = True              neg : pos    =      7.1 : 1.0
         contains(kudos) = True              pos : neg    =      6.5 : 1.0
        contains(shoddy) = True              neg : pos    =      6.4 : 1.0
        contains(justin) = True              neg : pos    =      5.8 : 1.0
     contains(underwood) = True              neg : pos    =      5.7 : 1.0
        contains(canyon) = True              neg : pos    =      5.7 : 1.0
  contains(surveillance) = True              neg : pos    =      5.7 : 1.0
       contains(unravel) = True              pos : neg    =      5.6 : 1.0

5. Select one of the classification tasks described in this chapter, such as name
gender detection, document classification, part-of-speech tagging, or dialogue act
classification. Using the same training and test data, and the same feature extractor,
build three classifiers for the task: a decision tree, a naive Bayes classifier, and a
Maximum Entropy classifier. Compare the performance of the three classifiers on
your selected task. How do you think that your results might be different if you
used a different feature extractor?

In [ ]:
def gender_features(word):
    return {'last_letter'  : word[-1],
            'length'       : len(word),
            'first_letter' : word[0],
            'last_2letters': word[-2:],
            'first_2letters': word[:2],
            'number_vowels': sum([1 for ch in word if ch in 'AEIOUaeiouy']),
            'first_vowel'  : [i for i in range(len(word)) 
                              if word[i] in 'AEIOUaeiouy'][0],
            'double_letters': sum([1 for ch in range(len(word) - 1)
                                  if word[ch] == word[ch + 1]])}

In [ ]:
from nltk.corpus import names
import random

labeled_names = ([(name, 'male') for name in names.words('male.txt')] + 
                 [(name, 'female') for name in names.words('female.txt')])
random.shuffle(labeled_names)

In [ ]:

featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, devtest_set, test_set = featuresets[1000:], featuresets[500:1000], featuresets[:500]
classifier = nltk.classify.MaxentClassifier.train(train_set, max_iter = 50)

  ==> Training (50 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.374
             2          -0.48746        0.737
             3          -0.41834        0.807
             4          -0.37851        0.817
             5          -0.35346        0.823
             6          -0.33646        0.826
             7          -0.32422        0.826
             8          -0.31501        0.827
             9          -0.30784        0.829
            10          -0.30209        0.830
            11          -0.29737        0.831
            12          -0.29343        0.833
            13          -0.29009        0.834
            14          -0.28721        0.835
            15          -0.28470        0.836
            16          -0.28250        0.837
            17          -0.28054        0.837
            18          -0.27879        0.838
            19          -0.27722        0.838
  

In [ ]:
classifier = nltk.DecisionTreeClassifier.train(train_set)

nltk.classify.accuracy(classifier, test_set)

0.734

In [ ]:

classifier = nltk.NaiveBayesClassifier.train(train_set)

nltk.classify.accuracy(classifier, test_set)

0.78